In [73]:
import pandas as pd
import numpy as np

In [111]:
# Finds events with multiple pulses at the same sensors
# here we ask for at least num_min_total_repeats repetitions
num_min_total_repeats = 10
for i in range(1000):
    unique_indices = np.unique(df.index)
    pulses = np.array(df.loc[unique_indices[i]])
    if(pulses[:,0].shape[0] - np.unique(pulses[:,0]).shape[0] >= num_min_total_repeats):
        print(unique_indices[i])

335176937
335176943
335176960
335176999
335177001
335177018


KeyboardInterrupt: 

In [ ]:
# Without considerations for the auxiliary label and no explicit features consisting of sensor geometry

# Class for a dataset generated from a dataframe and data from the sensor geometry file
class NeutrinoDataset():
    def __init__(self, batch_df, sg):
        self.sensor_geom = sg
        self.dataframe = batch_df
        sensor_loc = np.array(sg.iloc[:])[:, 1:]
        self.num_features = 5160*3
        self.num_events = df.index.nunique()
    
    # Replaces sensor ID with sensor coordinates 
    def event_data(self, i):
        df = self.dataframe
        sg = self.sensor_geom
        pulse_array = np.array(df.loc[df.index[i]])
        pulse_array_sensors = np.concatenate((np.expand_dims(np.arange(5160), axis=1), np.zeros([5160, 3])), 1)

        for pulse in pulse_array:
            if(pulse_array_sensors[pulse[0]][1] == 0):
                pulse_array_sensors[pulse[0]][1] = pulse[1] # first time
            else:
                # possible last time, will be the last time for the actual last one
                pulse_array_sensors[pulse[0]][2] = pulse[1] 
            # Add charge
            pulse_array_sensors[pulse[0]][3] += pulse[2]
                
        return np.flatten(pulse_array_sensors[:, 1:])

In [ ]:
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)
import torch
import torch.nn as nn
from torch.nn import Linear

sg = pd.read_csv('sensor_geometry.csv')
batch_df = pd.read_parquet('batch_104.parquet')

dataset = NeutrinoDataset(batch_df, sg)

class NNPredictor(torch.nn.Module):
    def __init__(self, use_activation = 'relu'):
        super().__init__()
        torch.manual_seed(1234)
        self.layers = nn.ModuleList()
        self.layer_norms = nn.ModuleList()
    
        self.layers.append(nn.Linear(dataset.num_features, 100))
        self.layers.append(nn.Linear(100, 50))
        self.layers.append(nn.Linear(50, 10))
        self.classifier = (nn.Linear(10,1))

    def forward(self, x, edge_index):
        new_x = x
        if(use_activation):
            for layer in self.layers:
                new_x = layer(new_x)
                new_x = nn.ReLU()(new_x)
        else:
            for layer in self.layers:
                new_x = layer(new_x)
        
        # Apply a final (linear) classifier.

        return self.classifier(new_x)

model = NNPredictor()
print(model)